In [2]:
%matplotlib tk

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os

# Collecting Corresponding Points from images
### we will use:
- ginput(n)
- numpy.save(path, var) - save variable into binary to path
- numpy.load(path) - load that saved variable

In [4]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os

def pick_and_save_correspondences(im1_path, im2_path, n_corresp = 4, sample_name = 'temp' ):
    # Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(17, 14))
# collecting correspondances
    # Load or create two example images
    img1 = mpimg.imread(im1_path)
    img2 = mpimg.imread(im2_path)

    ax1.imshow(img1)
    ax1.set_title("Image 1")

    ax2.imshow(img2)
    ax2.set_title("Image 2")

    plt.tight_layout()


    # Number of points to pick
    pts1 = []
    pts2 = []
    correspondances = []
    # Pick points alternately
    for i in range(n_corresp):
        # take point 1 at img1
        print(f"Select point {i+1} in Image 1")
        plt.sca(ax1)
        p1 = plt.ginput(1)[0]
        plt.plot(p1[0], p1[1], 'ro')
        pts1.append(p1)

        # take point 2 at img2
        print(f"Select corresponding point {i+1} in Image 2")
        plt.sca(ax2)
        p2 = plt.ginput(1)[0]
        plt.plot(p2[0], p2[1], 'go')
        pts2.append(p2)

        # push correspondance to list
        correspondances.append((p1, p2))

        # Draw lines showing correspondence across subplots
        # Convert axes coordinates to normalised figure coordinates
        fig.canvas.draw()
        transFigure = fig.transFigure.inverted()
        coord1 = ax1.transData.transform(p1)
        coord2 = ax2.transData.transform(p2)
        fig_coord1 = transFigure.transform(coord1)
        fig_coord2 = transFigure.transform(coord2)
        line = plt.Line2D((fig_coord1[0], fig_coord2[0]),
                        (fig_coord1[1], fig_coord2[1]),
                        transform=fig.transFigure,
                        color='yellow', linestyle='--')
        fig.lines.append(line)
        plt.draw()
    
    plt.close()

    # saving selected points
    output_dir = f'./homography/{sample_name}/samples'
    os.makedirs(output_dir, exist_ok=True)
    
    output_fullpath = output_dir + f'/{n_corresp}_corresp'
    np.save(output_fullpath, correspondances)


### Plotting funs used

>We have **figure coordinate system (display)** that contains subplots, etc. (`fig`).
>Every figure is like a **canvas**.

---

#### 1️⃣ `fig.transFigure.inverted()` vs `fig.transFigure`

* **Display (figure) coordinate system** can be:

1. **Normalized** `(0,0)` to `(1,1)`

   * Independent of the figure size → `fig.transFigure` (it's just a coordinate system)
   * **MAPping:** `FIG.NORMAL_COORD → FIG.PIXEL_COORD` → `fig.transFigure.transform(coords)`
   * **MAPping:** `FIG.PIXEL_COORD → FIG.NORMAL_COORD` → `fig.transFigure.inverted().transform(coords)`

2. **In pixels**

   * Shows actual pixels on the screen
   * Depends on the size of the figure window (canvas) → `fig.transFigure.inverted()` (it's just coord system for fig in pixels)

---

#### 2️⃣ `ax_i.transData.transform(x, y)`

1. `ax1.transData.transform(x, y)` maps **data points** `(x, y)` on subplot `ax1` to **pixels** in display coordinates (relative to the figure canvas).
2. Similarly, `ax2.transData` does the same for subplot `ax2`.

---

#### 3️⃣ Why do we map clicked points up until getting normalized figure coordinates?

* Because we **cannot use `fig.transFigure.inverted` as the transform coordinate system** for drawing lines.

---


In [34]:
# collecting correspondances
im1_path = './images/paris/paris_c.jpg'
im2_path = './images/paris/paris_b.jpg' # (BASE IMG)

#result_path = result_root_dir + os.path.basename(im1_path)
for sample_size in [4,6,8,10, 12, 15]:
    pick_and_save_correspondences(im1_path, im2_path, sample_size, sample_name='paris/H_cb')

Select point 1 in Image 1
Select corresponding point 1 in Image 2
Select point 2 in Image 1
Select corresponding point 2 in Image 2
Select point 3 in Image 1
Select corresponding point 3 in Image 2
Select point 4 in Image 1
Select corresponding point 4 in Image 2
Select point 1 in Image 1
Select corresponding point 1 in Image 2
Select point 2 in Image 1
Select corresponding point 2 in Image 2
Select point 3 in Image 1
Select corresponding point 3 in Image 2
Select point 4 in Image 1
Select corresponding point 4 in Image 2
Select point 5 in Image 1
Select corresponding point 5 in Image 2
Select point 6 in Image 1
Select corresponding point 6 in Image 2
Select point 1 in Image 1
Select corresponding point 1 in Image 2
Select point 2 in Image 1
Select corresponding point 2 in Image 2
Select point 3 in Image 1
Select corresponding point 3 in Image 2
Select point 4 in Image 1
Select corresponding point 4 in Image 2
Select point 5 in Image 1
Select corresponding point 5 in Image 2
Select poi

In [3]:
sample_path = './homography/cmpe_building/left1_left2/samples/4_corresp.npy'

def homography_by_sample(sample_path):
    # load saved sample info
    points_data = np.load(sample_path)

    # compute homography
    points_im1 = points_data[:, 0]
    points_im2 = points_data[:, 1]

    return computeH(points_im1, points_im2)

def computeH(points_im1, points_im2):

    # constructing CALIBRATION MATRIX
    X1 = points_im1[:, 0]
    Y1 = points_im1[:, 1]

    ones = np.ones_like(X1)
    zeros = np.zeros_like(X1)

    X2 = points_im2[:, 0]
    Y2 = points_im2[:, 1]

    # last 3 columns of rows of type-1
    NX2X1 = -1 * X2 * X1
    NX2Y1 = -1 * X2 * Y1
    NX2 = -1 * X2

    # last 3 columns of other rows
    NY2X1 = -1 * Y2 * X1
    NY2Y1 = -1 * Y2 * Y1
    NY2 = -1 * Y2

    row1 = np.stack([X1, Y1, ones, zeros, zeros, zeros, NX2X1, NX2Y1, NX2], axis = 1)
    row2 = np.stack([zeros, zeros, zeros, X1, Y1, ones, NY2X1, NY2Y1, NY2], axis = 1)

    A_matrix = np.vstack([row1, row2])

    # compute svd for minimisation problem: h = argmin_h(Ah)
    U, S, Vt = np.linalg.svd(A_matrix)
    # solution is last row of V
    h = Vt[-1, :]
    H = h.reshape(3,3)

    # normalised solution to right-bottom corner element
    H = (H / H[2,2])

    return H

In [68]:
sample_sizes = [4, 6, 8, 10, 12, 15]

for sample_size in sample_sizes:
    sample_path = f'./homography/cmpe_building/left1_left2/samples/{sample_size}_corresp.npy'
    print(homography_by_sample(sample_path))

[[ 5.77659417e-01 -7.56560787e-02  3.54225652e+02]
 [-1.12237382e-01  7.56285536e-01  8.90783837e+01]
 [-2.77296245e-04 -5.94592452e-05  1.00000000e+00]]
[[ 8.02261996e-01 -1.77854556e-02  2.68929237e+02]
 [-7.32107906e-02  9.08446409e-01  4.36167700e+01]
 [-1.62868120e-04 -1.37430832e-05  1.00000000e+00]]
[[ 7.53844199e-01 -4.02178133e-04  2.79165644e+02]
 [-7.99800681e-02  8.90470781e-01  4.83669598e+01]
 [-1.95611409e-04  1.22147475e-06  1.00000000e+00]]
[[ 7.28836195e-01 -2.06233652e-02  2.95480719e+02]
 [-8.58860833e-02  8.72984640e-01  5.53838440e+01]
 [-2.01520192e-04 -1.46467713e-05  1.00000000e+00]]
[[ 7.81663710e-01 -2.18801081e-02  2.71836733e+02]
 [-7.19569774e-02  8.97486036e-01  4.06820023e+01]
 [-1.74722084e-04 -2.43652027e-05  1.00000000e+00]]
[[ 7.68419393e-01 -4.08229185e-02  2.83013409e+02]
 [-7.48318135e-02  8.81721867e-01  4.66858606e+01]
 [-1.76505839e-04 -3.78310602e-05  1.00000000e+00]]


## 🌀 Warping Image

### 0) Why not Forward Warping?

We **don’t use forward warping** because it’s inefficient and produces artifacts.
Since real images contain **noise** and **picked correspondence points aren’t perfectly accurate**,
some points in `img1` may get mapped to the **same pixel** in `img2` plane by the homography $ H_{12} $.

As a result, when we compute

$
(x', y', 1)^T = H_{12} \cdot (x, y, 1)^T
$

the set of mapped pixels ${(x', y')}$ may **not cover every pixel** in the target image —
causing **holes** (unfilled regions) where no source pixel lands.

---

### 1) Backward Warping Idea

The idea is simple:
we **invert the mapping** and instead ask:

> “For each pixel (x′, y′) in the output image (img2 plane),
> from which location (x, y) in img1 should its color come?”

Mathematically:

$
\text{color}(x', y') = \text{color}*{\text{img1}}(H*{12}^{-1} \cdot (x', y', 1)^T)
$

So for every output pixel, we **trace it backward** through $ H_{12}^{-1} $
to find where it came from in `img1`.

---

### 1.1) Constructing All (x′, y′, 1) Points

To apply $ H_{12}^{-1} $ to **all output pixels at once**,
we create a grid of all pixel coordinates on the output (img2) plane.

We use **`np.meshgrid`** to generate all $(x′, y′)$ combinations:

Perfect — here’s a clear explanation (with math) of **how `np.meshgrid` builds the full coordinate matrix** used in backward warping 👇

---

## 🧩 Constructing the Coordinate Matrix with `np.meshgrid`

Let’s say your **output image** (the plane we warp onto) has size:

$
\text{height} = h_{out}, \quad \text{width} = w_{out}
$

So, pixel coordinates on this image are integer grid points:

$
x' \in [0, w_{out}-1], \quad y' \in [0, h_{out}-1]
$

---

### 1️⃣ Build coordinate grids

We use:

```python
xs, ys = np.meshgrid(np.arange(w_out), np.arange(h_out))
```

* `np.arange(w_out)` → `[0, 1, 2, ..., w_out-1]`  → **x-coordinates**
* `np.arange(h_out)` → `[0, 1, 2, ..., h_out-1]` → **y-coordinates**
* `meshgrid` repeats them to form **full 2D coordinate grids**:

Example (for `w_out=3`, `h_out=2`):

```python
xs =
[[0, 1, 2],
 [0, 1, 2]]

ys =
[[0, 0, 0],
 [1, 1, 1]]
```

That means:

* first row of `xs, ys` corresponds to y=0 (top row of image)
* second row corresponds to y=1, etc.
Together they define a mesh of coordinate pairs:

```scss
    (0,0) (1,0) (2,0)
    (0,1) (1,1) (2,1)
```

---

### 2️⃣ Flatten into column form

We flatten both to 1D arrays:

```python
x_flat = xs.ravel()
y_flat = ys.ravel()
```

Result:

$
x_{flat} = [0, 1, 2, 0, 1, 2]
$
$
y_{flat} = [0, 0, 0, 1, 1, 1]
$

Each pair ((x_i', y_i')) corresponds to **one pixel** in the output image.

---

### 3️⃣ Convert to homogeneous coordinates

Add a third coordinate = 1 for all points:

```python
ones = np.ones_like(x_flat)
```

Then **stack** them as rows (axis=0):

```python
p2_h = np.stack([x_flat, y_flat, ones], axis=0)
```

---

### 4️⃣ Resulting matrix shape

`p2_h` is a **3×N matrix**, where $ N = h_{out} \times w_{out} $.

$
p_2^h =
\begin{bmatrix}
x'_1 & x'_2 & \dots & x'_N \\
y'_1 & y'_2 & \dots & y'_N \\
1 & 1 & \dots & 1
\end{bmatrix}
$

Each column corresponds to **where img2 pixel comes from, if it was on plane img1**.
---

### 5️⃣ Why this form?

This form allows us to apply the inverse homography to *all pixels at once* using matrix multiplication:

$
p_1^h = H_{12}^{-1} \cdot p_2^h
$

which efficiently gives the mapped coordinates ((x, y)) in the source image for **every output pixel**.

---

---

## Interpolating and Color setting at once

### 1️⃣ Ideal mathematical model (no noise, integer coordinates)

If your homography were **perfect** and pixel-aligned, then for each pixel ((x',y')) on the **output image (img2 plane)**,
you’d find its corresponding pixel ((x,y)) on **img1 plane** as:

$
[x, y, 1]^T ;=; H^{-1} [x', y', 1]^T
$

and ideally, those ((x,y)) would be **integers** (exact pixel centers).
So, the color mapping is:

$
\text{color}*{\text{out}}(x',y') ;=; \text{color}*{\text{img1}}(x,y)
$

---

### 2️⃣ Real-world case (noise, non-integer mappings) - OUR CASE

To get the color, we interpolate between the four neighboring pixels around ((x,y)):

$
\text{color}_{\text{img2}}(x',y') = \text{color}_{\text{img1}}\big( \text{Interpolation}(H^{-1} [x', y', 1]^T) \big)
$

---

### 🔍 Intuitive explanation

* $H^{-1}$ tells **where this pixel on the output came from** on img1.
* That source coordinate may not be an integer pixel → so we **interpolate** color from nearby pixels in img1.
* That’s why we say “backward warping”:
  we go **back** from output → input to **sample** colors smoothly.

---

In [4]:
def warp(image, homography, output_shape=None):
    h_in, w_in = image.shape[:2]

    if output_shape is None:
        # get sizes of image (it's img1)
        height = image.shape[0]
        width = image.shape[1]
    else:
        height, width = output_shape

    # get matrix of all points as columns
    x_axis = np.arange(width)
    y_axis = np.arange(height)

    # rows we will stack for the matrix
    x_mesh, y_mesh = np.meshgrid(x_axis, y_axis)
    ones = np.ones_like(np.ravel(x_mesh))
    
    # A is matrix of all points in homogeneos coord.sys for plane 2
    P2 = np.stack([np.ravel(x_mesh), np.ravel(y_mesh), ones], axis = 0)

    # now, we need to get colors of them by backward transform
    P21 = np.dot(np.linalg.inv(homography), P2 )
    P21 = P21 / P21[2,:]
    
    # interpolating with opencv function
    
    # 1. get coordinate maps
    x_map = P21[0, :].reshape(height, width).astype(np.float32)
    y_map = P21[1, :].reshape(height, width).astype(np.float32)

    # 2. using interpolation function opencv that also makes mapping of colors
    warped_image = cv2.remap(image, x_map, y_map ,interpolation = cv2.INTER_LINEAR, borderMode = cv2.BORDER_CONSTANT, borderValue=0)
    
    # return interpolated and remaped 
    return warped_image

## Making Warped Part Visible — `warp_full`

1. Apply transform so all warped corners have positive coordinates:  
   $$
   H_{\text{adjusted}} = 
   \begin{bmatrix}
   1 & 0 & -x_{\min}\\
   0 & 1 & -y_{\min}\\
   0 & 0 & 1
   \end{bmatrix} H
   $$
   ensuring $ H_{\text{adjusted}}^{-1}(x_b, y_b, 1) = (x, y, 1) $ lies in image grid.  

2. Set output size to  
   $$
      W = \lceil x_{\max} - x_{\min} \rceil, \quad 
      H = \lceil y_{\max} - y_{\min} \rceil,
   $$
   so all backward-transformed points fit within the visible output.


In [5]:

def warp_full(image, homography):
    h, w = image.shape[:2]

    # Find corners and warp them
    corners = np.array([[0, 0, 1],
                        [w, 0, 1],
                        [w, h, 1],
                        [0, h, 1]]).T
    warped_corners = homography @ corners
    warped_corners /= warped_corners[2, :]
    xs, ys = warped_corners[0], warped_corners[1]

    # Translate original corners to BASE img. plane
    xmin, xmax = xs.min(), xs.max()
    ymin, ymax = ys.min(), ys.max()

    # Get Translation matrix, making adjusted Homography
    trans = np.array([[1, 0, -xmin],
                      [0, 1, -ymin],
                      [0, 0, 1]])
    
    # H_adj_inverse will now, make base coord. to map at coorners of original img.
    # So that in Backward Transform. we will not have points outside original image plane.
    H_adj = trans @ homography

    out_w, out_h = int(np.ceil(xmax - xmin)), int(np.ceil(ymax - ymin))
    return warp(image, H_adj, (out_h, out_w))

## Task Implementation

### Task 1 — Paris

Steps:  
- [ ] (1) Compute homography \(H\) using correspondences: **paris_a → paris_b (base)**.  
- [ ] (2) Warp and stitch **{paris_a, paris_b, paris_c}** onto the **paris_b** base frame.


In [ ]:
# 1. Stitching Paris Images, with paris_b as base.
base_path =  './tasks/1'


img_paris_a = mpimg.imread('./images/paris/paris_a.jpg')
img_paris_b = mpimg.imread('./images/paris/paris_b.jpg')
img_paris_c = mpimg.imread('./images/paris/paris_c.jpg')

# warping for all H_cb samples
for sample_size in [4, 6, 8, 10, 12, 15]:
    for original_img, original_img_name in [[img_paris_a, 'paris_a'], [img_paris_b, 'paris_b']]:
        # getting calculated homographies by sample_size (we pick 15)
        if original_img_name == 'paris_a':
            homography = homography_by_sample(f'./homography/paris/H_ab/samples/{sample_size}_corresp.npy')
        else:
            homography = homography_by_sample(f'./homography/paris/H_cb/samples/{sample_size}_corresp.npy')
        
        # warping with img size fitting
        warped_img = warp_full(original_img, homography)

        # saving result
        result_save_path = base_path + f'/{original_img_name}/samples/{sample_size}_corresp/'
        os.makedirs(result_save_path, exist_ok=True )
        mpimg.imsave(result_save_path + f'warped_{original_img_name}(base_b).jpg', warped_img)



    

ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

## Task 2

In [10]:
for building_name in ['cmpe_building', 'north_campus']:
    # build full dir-path
    base_path = './tasks/2/{building_name}'

    #2.1 left-to-right
    img_dir = f'{base_path}/left_to_right/'

    #2.2 middle out
    img_dir = f'{base_path}/middle_out/'

    #2.3 first_out_then_middle
    img_dir = f'{base_path}/first_out_then_middle/'

    # Stitching Image